In [ ]:
import os

In [ ]:
mind_data_path = "/kaggle/input/mind-data/mind_data"  
os.makedirs(mind_data_path, exist_ok=True) 

In [ ]:
!pip install tensorflow==2.15.0 "numpy<2.0"

In [ ]:
# Cài cuDNN 8.9 từ pip
!pip uninstall tensorflow -y
!pip install nvidia-cudnn-cu12==8.9.7.29
!pip install tensorflow==2.15.0

# Thiết lập LD_LIBRARY_PATH
import os
cudnn_path = '/usr/local/lib/python3.11/dist-packages/nvidia/cudnn/lib'
os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

print("✅ cuDNN 8.9 installed. Please RESTART KERNEL!")

In [ ]:
import os
cudnn_path = '/usr/local/lib/python3.11/dist-packages/nvidia/cudnn/lib'
os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))

In [ ]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

import tensorflow as tf
from tensorflow.keras.layers import Embedding as KEmbedding  # lấy class gốc

# Nếu đã patch trước đó (trong cùng session), restore về hàm gốc Keras
tf.keras.layers.Embedding.__init__ = KEmbedding.__init__


tf.keras.layers.Embedding._true_init = KEmbedding.__init__

def patched_embedding_init(self, *args, **kwargs):
    with tf.device("/GPU:0"):
        tf.keras.layers.Embedding._true_init(self, *args, **kwargs)

tf.keras.layers.Embedding.__init__ = patched_embedding_init
print("✅ Patched Embedding (safe v2, no recursion)")


In [ ]:
lstur_yaml = """
data:
  title_size: 30
  his_size: 50
  body_size: 50
  cat_num: 17
  subcat_num: 249
  data_format: news
  npratio: 4

model:
  # ===== required by your recommenders version =====
  filter_num: 300
  window_size: 3
  cnn_activation: relu

  # ===== shared / existing =====
  attention_hidden_dim: 300
  word_emb_dim: 300
  cat_emb_dim: 100
  subcat_emb_dim: 100
  dropout: 0.2

  # ===== keep these (some versions still expect them) =====
  head_num: 20
  head_dim: 20

  # ===== switch to LSTUR =====
  model_type: lstur
  gru_unit: 300
  user_emb_dim: 50
  type: con   # ini or con

training:
  batch_size: 128
  epochs: 10
  learning_rate: 0.0005
  loss: cross_entropy_loss
  optimizer: adam
  support_quick_scoring: true

info:
  metrics: ["group_auc", "mean_mrr", "ndcg@5;10"]
  show_step: 100000

files:
  wordEmb_file: "utils/embedding.npy"
  wordDict_file: "utils/word_dict.pkl"
  userDict_file: "utils/uid2index.pkl"
"""

with open("/kaggle/working/lstur.yaml", "w") as f:
    f.write(lstur_yaml)

print("✅ wrote /kaggle/working/lstur.yaml")


In [ ]:
# Cell 2: Setup
from recommenders.models.newsrec.newsrec_utils import prepare_hparams  
from recommenders.models.newsrec.models.lstur import LSTURModel  
from recommenders.models.newsrec.io.mind_iterator import MINDIterator  

train_news_file = os.path.join(mind_data_path, "train", "news.tsv")  
train_behaviors_file = os.path.join(mind_data_path, "train", "behaviors.tsv")  
valid_news_file = os.path.join(mind_data_path, "valid", "news.tsv")  
valid_behaviors_file = os.path.join(mind_data_path, "valid", "behaviors.tsv")  
wordEmb_file = os.path.join(mind_data_path, "utils", "embedding.npy")  
userDict_file = os.path.join(mind_data_path, "utils", "uid2index.pkl")  
wordDict_file = os.path.join(mind_data_path, "utils", "word_dict.pkl")  
yaml_file = "/kaggle/working/lstur.yaml"
print("Using yaml:", yaml_file)

hparams = prepare_hparams(  
    yaml_file,  
    wordEmb_file=wordEmb_file,  
    wordDict_file=wordDict_file,  
    userDict_file=userDict_file,  
    epochs=20,
    batch_size=128
)  
hparams.metrics = ["mean_mrr", "ndcg@5", "ndcg@10"]
hparams.early_stop_metric = "ndcg@5"
hparams.early_stop_patience = 3
hparams.show_step = 50
hparams.learning_rate = 1e-4
# Create model with GPU
with tf.device('/GPU:0'):
    iterator = MINDIterator  
    model = LSTURModel(hparams, iterator, seed=42)

print("✅ LSTURModel created on GPU")

In [ ]:
import os, re, glob

CKPT_DIR = "/kaggle/working/checkpoints_lstur"
os.makedirs(CKPT_DIR, exist_ok=True)

def ckpt_path(epoch: int) -> str:
    # Keras save_weights sẽ tạo ra file theo prefix, nên ta dùng prefix rõ ràng
    return os.path.join(CKPT_DIR, f"lstur_epoch_{epoch:03d}")

def save_ckpt(epoch: int):
    path = ckpt_path(epoch)
    model.model.save_weights(path)
    print(f"✅ Saved checkpoint: {path}")

def list_ckpts():
    # save_weights tạo ra nhiều file (vd .index/.data) hoặc 1 prefix tuỳ backend
    # ta list theo prefix
    prefixes = sorted(set(
        p.replace(".index","").replace(".data-00000-of-00001","")
        for p in glob.glob(os.path.join(CKPT_DIR, "lstur_epoch_*"))
    ))
    return prefixes

def latest_ckpt():
    prefixes = list_ckpts()
    if not prefixes:
        return None, 0
    # lấy epoch lớn nhất từ tên
    last = prefixes[-1]
    m = re.search(r"lstur_epoch_(\d+)$", last)
    ep = int(m.group(1)) if m else 0
    return last, ep


In [ ]:
#Training
print("🔥 Starting training...")
hparams.epochs = 1
NUM_EPOCHS = 10 

for ep in range(1, NUM_EPOCHS + 1):
    print(f"\n===== EPOCH {ep}/{NUM_EPOCHS} =====")
    
    model.fit(
        train_news_file,
        train_behaviors_file,
        valid_news_file,
        valid_behaviors_file,
    )
    
    # (tuỳ chọn) eval để theo dõi metric
    eval_results = model.run_eval(valid_news_file, valid_behaviors_file)
    print("Eval:", eval_results)
    
    # lưu checkpoint epoch này
    save_ckpt(ep)

In [ ]:
model = LSTURModel(hparams, MINDIterator, seed=42)

ckpt, last_ep = latest_ckpt()
print("Latest:", ckpt, "epoch:", last_ep)

if ckpt is not None:
    model.model.load_weights(ckpt)
    print(f"✅ Loaded checkpoint epoch {last_ep}")
else:
    print("⚠️ No checkpoint found, training from scratch.")


In [ ]:
# Đánh giá trên validation set  
eval_results = model.run_eval(valid_news_file, valid_behaviors_file)  
print("Kết quả đánh giá:")  
for metric, value in eval_results.items():  
    print(f"{metric}: {value:.4f}")

In [ ]:
model_path = os.path.join("/kaggle/working/", "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

In [ ]:
from tqdm import tqdm
import numpy as np

test_behaviors_file = "/kaggle/input/mind-data/mind_data/test/behaviors.tsv"  
test_news_file = "/kaggle/input/mind-data/mind_data/test/news.tsv"

print("🔧 Patching iterator for test set (no labels)...")

# Backup original method
original_init = model.test_iterator.init_behaviors

def init_behaviors_no_labels(behaviors_file):
    """Modified init_behaviors for test set without labels"""
    model.test_iterator.histories = []
    model.test_iterator.imprs = []
    model.test_iterator.labels = []
    model.test_iterator.impr_indexes = []
    model.test_iterator.uindexes = []

    with open(behaviors_file, "r", encoding="utf-8") as rd:
        impr_index = 0
        for line in rd:
            uid, time, history, impr = line.strip("\n").split("\t")[-4:]

            # Parse history
            history = [model.test_iterator.nid2index[i] for i in history.split() if i in model.test_iterator.nid2index]
            history = [0] * (model.test_iterator.his_size - len(history)) + history[:model.test_iterator.his_size]

            # Parse impressions - TEST SET KHÔNG CÓ LABEL
            impr_news = []
            for item in impr.split():
                # Test set: chỉ có news_id, KHÔNG có "-0" hay "-1"
                if "-" in item:
                    # Validation/train set format: N12345-1
                    news_id = item.split("-")[0]
                else:
                    # Test set format: N12345
                    news_id = item
                
                if news_id in model.test_iterator.nid2index:
                    impr_news.append(model.test_iterator.nid2index[news_id])
            
            # Tạo dummy labels (không dùng cho test)
            label = [0] * len(impr_news)
            
            uindex = model.test_iterator.uid2index[uid] if uid in model.test_iterator.uid2index else 0

            model.test_iterator.histories.append(history)
            model.test_iterator.imprs.append(impr_news)
            model.test_iterator.labels.append(label)
            model.test_iterator.impr_indexes.append(impr_index)
            model.test_iterator.uindexes.append(uindex)
            impr_index += 1

# Apply patch
model.test_iterator.init_behaviors = init_behaviors_no_labels

print("✅ Iterator patched for test set")

# Run evaluationa
print("\n🔍 Running evaluation on test set...")
print("   Model will use trained weights to generate rankings")

group_impr_indexes, group_labels, group_preds = model.run_fast_eval(
    test_news_file, test_behaviors_file
)

print(f"\n✅ Generated predictions for {len(group_impr_indexes)} impressions")

# Write predictions
print("\n💾 Writing predictions to file...")
prediction_file = "/kaggle/working/prediction.txt"

with open(prediction_file, 'w') as f:  
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds), 
                                   total=len(group_impr_indexes),
                                   desc="Writing"):  
        # MIND competition format: impression_id bắt đầu từ 1
        impr_id = impr_index + 1
        
        # Calculate rankings (score cao nhất = rank 1)
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()  
        pred_rank_str = '[' + ','.join([str(i) for i in pred_rank]) + ']'  
        
        f.write(f"{impr_id} {pred_rank_str}\n")

print(f"\n✅ Prediction file saved to: {prediction_file}")

# Validate output
print("\n🔍 Validating output format...")
with open(prediction_file, 'r') as f:
    lines = f.readlines()
    print(f"   Total predictions: {len(lines)}")
    print(f"   First 3 lines:")
    for i in range(min(3, len(lines))):
        parts = lines[i].strip().split()
        print(f"      ImprID={parts[0]}, Rankings={parts[1][:50]}{'...' if len(parts[1]) > 50 else ''}")

print("\n✅ File ready to submit to MIND competition!")
print(f"   Download: {prediction_file}")